In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
from chart_studio.plotly import iplot
from chart_studio import tools
from plotly.graph_objs import *
tools.set_credentials_file(username='ShawnXiao',api_key='iPcpCcCFROg5WfytqeAv')
import networkx as nx
import math
import igraph as ig
import plotly.offline as pyoff

### Data Load


In [2]:
score_df_path = '../data/SME_dataset.csv'
connect_flag_df_path = '../result/connect_flag_df_threshold_%s.csv'
factor_matrix_flag_df_path='../result/fator_matrix_threshold_%s.csv'

threshold_list = [0.05, 0.1]


In [12]:
rawdata_df=pd.read_csv(score_df_path)

threshold=threshold_list[0]    #plot result under threshold=0.05

factor_matrix_flag_df_path = factor_matrix_flag_df_path %(str(threshold))
factor_matrix_flag_df=pd.read_csv(factor_matrix_flag_df_path, index_col=0)

TypeError: not all arguments converted during string formatting

In [13]:
factor_matrix_flag_df=pd.read_csv(factor_matrix_flag_df_path, index_col=0)

In [14]:
cur_connect_flag_df_path = connect_flag_df_path % (str(threshold))
connect_flag_df = pd.read_csv(cur_connect_flag_df_path, index_col=0)
connect_def_num=sum((rawdata_df[connect_flag_df.values==1]['status']==1))  #connect and default
connect_act_num=sum((rawdata_df[connect_flag_df.values==1]['status']==0)) #connect and active

nonconnect_def_num=sum((rawdata_df[connect_flag_df.values==0]['status']==1))  #connect and default
nonconnect_act_num=sum((rawdata_df[connect_flag_df.values==0]['status']==0))  #connect and default

In [15]:
#factor_matrix_flag_df_path = factor_matrix_flag_df_path %(str(threshold))
factor_matrix_flag_df=pd.read_csv(factor_matrix_flag_df_path, index_col=0)

In [16]:
threshold=threshold_list[1]
factor_matrix_flag_df_path='../result/fator_matrix_threshold_0.1.csv'
# factor_matrix_flag_df_path = factor_matrix_flag_df_path %(str(threshold))
factor_matrix_flag_df1=pd.read_csv(factor_matrix_flag_df_path, index_col=0)


In [44]:
lomo1=factor_matrix_flag_df
factor_matrix_flag_df=factor_matrix_flag_df1

In [45]:
status_df=rawdata_df['status']
connect_default_df=factor_matrix_flag_df[(rawdata_df['status']==1)&(pd.DataFrame(connect_flag_df)['connect_flag']==1)]
connect_active_df=factor_matrix_flag_df[(rawdata_df['status']==0)&(pd.DataFrame(connect_flag_df)['connect_flag']==1)]
connect_df=pd.concat([connect_default_df,connect_active_df], axis=0)
connect_df

,0,1,2,3,4,5,6,7,8,9,...,15035,15036,15037,15038,15039,15040,15041,15042,15043,15044
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15036,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
15039,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
15041,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
15043,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,True,False,False,False,True,False


In [46]:
temp=np.array(connect_df.index).tolist()
temp=[str(x) for x in temp]
connect_df=connect_df[temp]

In [47]:
lomo_temp=np.tri(connect_df.shape[0],connect_df.shape[0],-1)
lomo_temp=lomo_temp==1
connect_df=connect_df&lomo_temp

In [48]:
edge_array=np.where(connect_df)
edge_array=np.array(edge_array).T

In [49]:
edge_array.shape

(391260, 2)

###  Function Definition

In [50]:
def generate_pos_circle(radius,pos_num):
    
    t = np.random.random(size=pos_num) * 2 * np.pi - np.pi
    x = np.cos(t)
    y = np.sin(t)
    i_set = np.arange(0,pos_num,1)
    for i in i_set:
        len = np.sqrt(np.random.random())
        x[i] = x[i] * len
        y[i] = y[i] * len
    x=x*radius
    y=y*radius
    pos_list=[x,y]
    return pos_list

In [51]:
def generate_pos_ring(r_inner,r_outer,pos_num):
    rad=np.sqrt(np.random.uniform(r_inner**2,r_outer**2,pos_num))
    angle=np.random.uniform(-np.pi,np.pi,pos_num)
    x,y=rad*np.cos(angle),rad*np.sin(angle)
    
    for i in range(x.shape[0]):
        if math.hypot(x[i],y[i])<r_inner:
            x[i],y[i]=-x[i],-y[i]
    return [x,y]

### Graph Plotting 

Canvas design

In [52]:
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

width=800
height=800

In [53]:
layout=Layout(title= "Coauthorship network of scientists working on network theory and experiment"+\
              "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=XAxis(axis),
    yaxis=YAxis(axis),
    margin=Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text='This igraph.Graph has the Kamada-Kawai layout',
            xref='paper',
            yref='paper',
            x=0,
            y=-0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ]
    )


D:\anaconda\lib\site-packages\plotly\graph_objs\_deprecations.py:550: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


D:\anaconda\lib\site-packages\plotly\graph_objs\_deprecations.py:578: DeprecationWarning:

plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis


D:\anaconda\lib\site-packages\plotly\graph_objs\_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




Plot Company Distribution

In [54]:
def generate_scatter(node_pos,size,color):
    N=len(node_pos[0])
    Xv=[node_pos[0][k] for k in range(N)]
    Yv=[node_pos[1][k] for k in range(N)]
    trace=Scatter(x=Xv,
               y=Yv,
               mode='markers',
               name='net',
               marker=dict(symbol='circle',
                             size=size, 
                             color=color,
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
         
                hoverinfo='text'
               )
    return trace

In [55]:
def generate_edge(def_pos,act_pos,edge_array):
    pos=np.vstack((np.array(def_pos).T,np.array(act_pos).T))
    Xed=[]
    Yed=[]
    for edge in edge_array[:100000]:
        Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
        Yed+=[pos[edge[0]][1],pos[edge[1]][1], None] 
    edge_link=Scatter(x=Xed,
               y=Yed,
               mode='lines',
               line=dict(color='rgb(150,150,150)', width=1),
               hoverinfo='none'
               )
    
    return edge_link

In [56]:
connect_default_node=generate_pos_circle(1,connect_def_num)
connect_active_node=generate_pos_circle(1,connect_act_num)
nonconnect_default_node=generate_pos_ring(1.5,2.5,nonconnect_def_num)
nonconnect_active_node=generate_pos_ring(1.5,2.5,nonconnect_act_num)

In [57]:
connect_def_scatter=generate_scatter(connect_default_node,5,'#CD3700')
connect_active_scatter=generate_scatter(connect_active_node,5,'#6959CD')
nonconnect_default_scatter=generate_scatter(nonconnect_default_node,5,'#CD3700')
nonconnect_active_scatter=generate_scatter(nonconnect_active_node,5,'#6959CD')
connect_vector=generate_edge(connect_default_node,connect_active_node,edge_array)

### draw network structure of all institutions

In [58]:
data=[connect_vector,connect_def_scatter,connect_active_scatter,nonconnect_default_scatter,nonconnect_active_scatter]  
# data=[connect_vector,connect_def_scatter,connect_active_scatter]
fig1=Figure(data=data,layout=layout)
pyoff.plot(fig1)

'temp-plot.html'

### draw network of connected component

In [59]:
data=[connect_vector,connect_def_scatter,connect_active_scatter]
fig1=Figure(data=data,layout=layout)
pyoff.plot(fig1)

'temp-plot.html'